In [ ]:
import os 
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold, content_types
from collections.abc import Iterable
from function_calls import final_factuality_factor_score, emotion_analyzer
from poli_stance_function_calling import perspective_analyzer
from google.protobuf.struct_pb2 import Struct


In [ ]:
api_key = os.getenv("API_KEY")
genai.configure(api_key=api_key)

Documentation: https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/function-calling/python.ipynb, https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb

In [ ]:
# tools = {
#   "functions": [
#       {
#         "name": "emotion_analyzer",
#         "description": "Analyzes the emotionality and exaggeration level in a given text and returns a scaled score.",
#         "parameters": {
#           "type": "object",
#           "properties": {
#             "text": {
#               "type": "string",
#               "description": "The text content to analyze for emotional intensity and exaggeration."
#             }
#           },
#           "required": ["text"]
#         }
#       },
#       {
#         "name": "final_factuality_factor_score",
#         "description": "Averages the microfactors from a single factuality factor. This function should be used when combining into an overall score.",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "microfactor_1": {
#                     "type": "float",
#                     "description": "First microfactor for a factuality factor, used to help calculate the factuality factor"
#                 },
#                 "microfactor_2": {
#                     "type": "float",
#                     "description": "Second microfactor for a factuality factor, used to help calculate the factuality factor"
#                 },
#                 "microfactor_3": {
#                     "type": "float",
#                     "description": "Third microfactor for a factuality factor, used to help calculate the factuality factor"
#                 }
#             },
#             "required": ["microfactor_1", "microfactor_2", "microfactor_3"]
#         }
#       }
#   ]
# }


### Sentiment function calling part

In [ ]:
generation_config = {
    "max_output_tokens": 4000, # less output, means faster
    "response_mime_type": "text/plain",
    "temperature": 1, # higher temp --> more risks the model takes with choices
    "top_p": 0.95, # how many tokens are considered when producing outputs
    "top_k": 40, # token is selected from 40 likely tokens
}

safety_settings = {
  HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

functions = {"final_factuality_factor_score": final_factuality_factor_score,
            "emotion_analyzer": emotion_analyzer,}

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-002",
    generation_config=generation_config,
    safety_settings=safety_settings,
    tools = functions.values(),
    system_instruction="""
        You are trying to fight against misinformation by scoring different articles on their factuality factors. 
        In your responses:
        - Use each function only once per request.
        - Integrate the results from the function calls to generate a complete response.
        - Do not assess an article until you are given a factuality factor to grade on.
        - Be concise and avoid redundant function calls.    
    """
)

In [ ]:
# chat = model.start_chat(enable_automatic_function_calling=True)

In [ ]:
# response = chat.send_message("""
# "Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!."
# Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:
# 1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.
# 2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.
# 3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.
# Then, combine them into an overall score for sensationalism and explain your thought process.
# Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once.
# """)
# print(f"Model response: {response.text}")


In [ ]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)

In [ ]:
tool_config = tool_config_from_mode("auto")
    

In [ ]:
def fc_generate_content(input):
    response = model.generate_content(input, tool_config=tool_config)
    parts = response.candidates[0].content.parts
    messages = []
    messages.append({"role": "user", "parts": [input]})
    for part in parts:
        if part.function_call:
            result = call_function(part.function_call, functions)
            s = Struct()
            s.update({"result": result})
            function_response = genai.protos.Part(
                function_response=genai.protos.FunctionResponse(name=part.function_call.name, response=s)
            )
            messages.append({"role": "model", "parts": [part]})
            messages.append({"role": "user", "parts": [function_response]})
        else:
            messages.append({"role": "model", "parts": [part.text]})
                # fmt: off
            #     {"role": "user",
            #     "parts": ["'Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!.'\
            # Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:\
            # 1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.\
            # 2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.\
            # 3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.\
            # Then, combine them into an overall score for sensationalism and explain your thought process.\
            # Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once."]},
            #     {"role": "model",
            #     "parts": response.candidates[0].content.parts},
            #     {"role": "user",
            #     "parts": [function_response]},
            #     # fmt: on
    print(messages)
    new_response = model.generate_content(messages)
    return new_response

In [ ]:
response = fc_generate_content("'Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!.'\
Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:\
1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.\
2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.\
3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.\
Then, combine them into an overall score for sensationalism and explain your thought process.\
Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once.")
# break down prompt and split into different intentions, intention: {}
# distinguish between each chunk, each chunk has a header 

In [ ]:
print(response.text)

### Political stance function calling

In [ ]:
generation_config = {
    "max_output_tokens": 4000, # less output, means faster
    "response_mime_type": "text/plain",
    "temperature": 1, # higher temp --> more risks the model takes with choices
    "top_p": 0.95, # how many tokens are considered when producing outputs
    "top_k": 40, # token is selected from 40 likely tokens
}

safety_settings = {
  HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

functions = {"final_factuality_factor_score": final_factuality_factor_score,
            "emotion_analyzer": emotion_analyzer,
            "perspective_analyzer": perspective_analyzer}

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-002",
    generation_config=generation_config,
    safety_settings=safety_settings,
    tools = functions.values(),
    system_instruction="""
        You are trying to fight against misinformation by scoring different articles on their factuality factors. 
        In your responses:
        - Use each function only once per request.
        - Integrate the results from the function calls to generate a complete response.
        - Do not assess an article until you are given a factuality factor to grade on.
        - Be concise and avoid redundant function calls.    
    """
)

In [ ]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)

In [ ]:
tool_config = tool_config_from_mode("auto")
    

In [ ]:
def fc_generate_content(input):
    response = model.generate_content(input, tool_config=tool_config)
    parts = response.candidates[0].content.parts
    messages = []
    messages.append({"role": "user", "parts": [input]})
    for part in parts:
        if part.function_call:
            result = call_function(part.function_call, functions)
            s = Struct()
            s.update({"result": result})
            function_response = genai.protos.Part(
                function_response=genai.protos.FunctionResponse(name=part.function_call.name, response=s)
            )
            messages.append({"role": "model", "parts": [part]})
            messages.append({"role": "user", "parts": [function_response]})
        else:
            messages.append({"role": "model", "parts": [part.text]})
                # fmt: off
            #     {"role": "user",
            #     "parts": ["'Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!.'\
            # Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:\
            # 1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.\
            # 2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.\
            # 3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.\
            # Then, combine them into an overall score for sensationalism and explain your thought process.\
            # Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once."]},
            #     {"role": "model",
            #     "parts": response.candidates[0].content.parts},
            #     {"role": "user",
            #     "parts": [function_response]},
            #     # fmt: on
    print(messages)
    new_response = model.generate_content(messages)
    return new_response

In [ ]:
response = fc_generate_content("'Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!.'\
Given the previous sentence, rate the sentence based on political stance with 1 being extremely biased towards democratic and \
10 being extremely biased towards conservative and 5 being neutral from a score of 1 to 10. \
First, rate the setence on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, \
ensuring that you explain the reasoning behind each score, and feel free to use function calling:\
1. Perspective Analysis: Identify underlying perspectives on issues or events. \
2. Bias evaluation: Evaluate if the stance is consistently biased. \
3. verify facts: Compare the stance against verified facts. This should be done paragraph by paragraph. \
Then, combine them into an overall score for a political stance. Lastly please phrase the overall score as 'Normal Prompting Overall Stance: {score}' \
, where score is calculated via the final_factuality_factor_score function. Each function call should be used once.")
# break down prompt and split into different intentions, intention: {}
# distinguish between each chunk, each chunk has a header 

In [ ]:
print(response.text)